In [27]:
import pandas as pd
import xlsxwriter


# Leer los DataFrames
data_frame_combinado_indep = pd.read_csv('../DataIntermediate/combined_data_indep_variables.csv', sep=',').dropna()
data_frame_combinado_dep = pd.read_csv('../DataIntermediate/combined_data_dep_variables.csv', sep=',').dropna()

# Verificar las columnas de cada DataFrame
print("Variables Independientes:\n", data_frame_combinado_indep.columns)
print("Variables Dependientes:\n", data_frame_combinado_dep.columns)


Variables Independientes:
 Index(['id', 'OutDeg_Var.Expl_RedConfliAguaTot',
       'Indeg_Var.Expl_RedConfliAguaTot',
       'Out2local_Var.Expl_RedConfliAguaTot',
       'In2local_Var.Expl_RedConfliAguaTot',
       'OutBetaCent_Var.Expl_RedConfliAguaTot',
       'InBetaCent_Var.Expl_RedConfliAguaTot',
       'Out2Step_Var.Expl_RedConfliAguaTot',
       'In2Step_Var.Expl_RedConfliAguaTot', 'OutARD_Var.Expl_RedConfliAguaTot',
       ...
       'Out2Step_Var.Explic_RedeEmprestParcel',
       'In2Step_Var.Explic_RedeEmprestParcel',
       'OutARD_Var.Explic_RedeEmprestParcel',
       'InARD_Var.Explic_RedeEmprestParcel',
       'OutClose_Var.Explic_RedeEmprestParcel',
       'InClose_Var.Explic_RedeEmprestParcel',
       'Between_Var.Explic_RedeEmprestParcel',
       '2StepBet_Var.Explic_RedeEmprestParcel',
       'Frag_Var.Explic_RedeEmprestParcel',
       'DwFrag_Var.Explic_RedeEmprestParcel'],
      dtype='object', length=203)
Variables Dependientes:
 Index(['id', 'Degree_Rede_Entreaju

In [32]:
data_frame_combinado_indep.head()


,id,OutDeg_Var.Expl_RedConfliAguaTot,Indeg_Var.Expl_RedConfliAguaTot,Out2local_Var.Expl_RedConfliAguaTot,In2local_Var.Expl_RedConfliAguaTot,OutBetaCent_Var.Expl_RedConfliAguaTot,InBetaCent_Var.Expl_RedConfliAguaTot,Out2Step_Var.Expl_RedConfliAguaTot,In2Step_Var.Expl_RedConfliAguaTot,OutARD_Var.Expl_RedConfliAguaTot,...,Out2Step_Var.Explic_RedeEmprestParcel,In2Step_Var.Explic_RedeEmprestParcel,OutARD_Var.Explic_RedeEmprestParcel,InARD_Var.Explic_RedeEmprestParcel,OutClose_Var.Explic_RedeEmprestParcel,InClose_Var.Explic_RedeEmprestParcel,Between_Var.Explic_RedeEmprestParcel,2StepBet_Var.Explic_RedeEmprestParcel,Frag_Var.Explic_RedeEmprestParcel,DwFrag_Var.Explic_RedeEmprestParcel
0,Aiace Quior,0,0,0,0,0.0,0.0,0,0,0.0,...,0,0,0.0,0.0,516,516,0,0,-0.000038,-0.000036
1,Alamada Bia,0,0,0,0,0.0,0.0,0,0,0.0,...,0,0,0.0,0.0,516,516,0,0,-0.000038,-0.000036
2,Alanghate Mfumba,0,0,0,0,0.0,0.0,0,0,0.0,...,0,0,0.0,0.0,516,516,0,0,-0.000038,-0.000036
3,Albat Calabus,0,0,0,0,0.0,0.0,0,0,0.0,...,0,0,0.0,0.0,516,516,0,0,-0.000038,-0.000036
4,Americano Mbunde,0,0,0,0,0.0,0.0,0,0,0.0,...,0,0,0.0,0.0,516,516,0,0,-0.000038,-0.000036


In [31]:
import pandas as pd
import statsmodels.api as sm
from openpyxl import Workbook

# Seleccionar todas las variables independientes
independent_vars = data_frame_combinado_indep.drop(columns=['id'])

# Añadir una constante (intercepto) al modelo
X = sm.add_constant(independent_vars)

# Diccionarios para almacenar los resultados
r2_scores = {}
p_values = {}

# Iterar sobre cada variable dependiente
for dependent_var in data_frame_combinado_dep.columns:
    if dependent_var != 'id':
        y = data_frame_combinado_dep[dependent_var]
        
        # Asegurarse de que no se realice la regresión de una variable contra sí misma
        if dependent_var not in independent_vars.columns:
            # Realizar la regresión OLS
            model = sm.OLS(y, X).fit()
            
            # Verificar si el R2 ajustado es válido
            if model.rsquared_adj != float('-inf') and not pd.isna(model.rsquared_adj):
                # Almacenar el R2 ajustado y los p-values
                r2_scores[dependent_var] = model.rsquared_adj
                p_values[dependent_var] = model.pvalues

# Convertir los diccionarios a dataframes
r2_df = pd.DataFrame(list(r2_scores.items()), columns=['Variable Dependiente', 'R2 Ajustado'])
p_values_df = pd.DataFrame(p_values).transpose()
p_values_df.reset_index(inplace=True)
p_values_df.columns = ['Variable Dependiente'] + list(p_values_df.columns[1:])

# Filtrar las variables con R2 ajustado superior a 0.30
filtered_r2_df = r2_df[r2_df['R2 Ajustado'] > 0.05]

# Crear un nuevo libro de trabajo de Excel
wb = Workbook()

# Hoja para R2
ws_r2 = wb.active
ws_r2.title = "R2 Ajustado"
for idx, row in r2_df.iterrows():
    ws_r2.cell(row=idx + 1, column=1, value=row['Variable Dependiente'])
    ws_r2.cell(row=idx + 1, column=2, value=row['R2 Ajustado'])

# Hoja para p-values
ws_pvalues = wb.create_sheet(title="P-Values")
for idx, row in p_values_df.iterrows():
    for col_idx, value in enumerate(row):
        ws_pvalues.cell(row=idx + 1, column=col_idx + 1, value=value)

# Hoja para las regresiones con R2 > 0.30
ws_filtered_r2 = wb.create_sheet(title="Regresiones con R2 > 0.30")
for idx, row in filtered_r2_df.iterrows():
    ws_filtered_r2.cell(row=idx + 1, column=1, value=row['Variable Dependiente'])
    ws_filtered_r2.cell(row=idx + 1, column=2, value=row['R2 Ajustado'])

# Guardar el archivo Excel
wb.save('../DataIntermediate/resultados_regresiones.xlsx')

print("Archivo 'resultados_regresiones.xlsx' guardado correctamente.")


c:\Users\cespe\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\regression\linear_model.py:1796: RuntimeWarning: divide by zero encountered in divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
c:\Users\cespe\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\regression\linear_model.py:1783: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


Archivo 'resultados_regresiones.xlsx' guardado correctamente.
